In [38]:
# NAO USAR A TABELA data_product, E SIM A store_product!!

In [39]:
import pandas as pd

In [40]:
df = pd.read_csv('inputs/data_product_202208091047.csv', usecols = ['id', 'barcode', 'name'])

In [41]:
df.shape

(69111, 3)

In [42]:
df['name'].nunique()


48879

In [43]:
df.drop_duplicates(subset = 'name', inplace = True)

In [44]:
df.shape

(48879, 3)

In [45]:
df['name'] = df['name'].astype(str)

In [46]:
df['name'] = df['name'].str.upper()

In [47]:
df['cleaned'] = df['name']

In [48]:
endswith_ = {
    'G':    -len('G'),
    'L':    -len('L'),
    'UN':   -len('UN'),
    'ML':   -len('ML'),
    'GR':   -len('GR'),
    'CPS':  -len('CPS'),
    'VOL':  -len('VOL'),
    'CAP':  -len('CAP')
}

In [49]:
def remove_endswith(name, numbers):
    return lambda words: words.replace(''.join([x for x in words.split(' ') if x.endswith(name) and x[:numbers].isdigit()]), '')

In [50]:
for k, v in endswith_.items():
    df['cleaned'] = df['cleaned'].apply(remove_endswith(k, v))

In [95]:
startswith_ = ['(', 'REF', 'C/', 'S/']

In [96]:
def remove_startswith(name):
    return lambda words: words.replace(''.join([x for x in words.split(' ') if x.startswith(name)]), '')

In [97]:
for name in startswith_:
    df['cleaned'] = df['cleaned'].apply(remove_startswith(name))

In [98]:
df['cleaned'] = df['cleaned'].str.strip()

In [99]:
df['num_words'] = df['cleaned'].apply(lambda x: len(str(x).split(' ')))

In [128]:
items_cleaned = [
    48, 57, 58, 60, 76, 77, 82, 83, 84, 88, 94, 102, 106, 116
]

In [129]:
df[df['id'].isin(items_cleaned)]

,id,barcode,name,cleaned,num_words
47,48,7896083409235,BIOSLIM PLUS 4 90CAP,BIOSLIM PLUS 4,3
56,57,7891721017377,CR GEL UNIVERSAL FPS20 45G,CR GEL UNIVERSAL FPS20,4
57,58,7891800348361,CURAT CREMER BARBIE 25UN,CURAT CREMER BARBIE,3
59,60,7896040704120,AMACIANTE VIDA MACIA 1L,AMACIANTE VIDA MACIA,3
75,76,7891142142979,EPIDRAT SENSI SH 120ML,EPIDRAT SENSI SH,3
76,77,7898064034801,ABS AFFECTIVE ELASTIC C/10 (FR10),ABS AFFECTIVE ELASTIC,3
81,82,7896025514126,ESC MARCO BONI REF7665T,ESC MARCO BONI,3
82,83,7896025508927,ESC MARCO BONI REF7951,ESC MARCO BONI,3
83,84,7898312327488,ESC PROART BAMBU REF 73252,ESC PROART BAMBU 73252,5
87,88,4015400151715,ABS TAMPAX COMPAK C/8 SUPER,ABS TAMPAX COMPAK SUPER,5


In [137]:
df.sort_values(by = 'cleaned', inplace = True)

In [ ]:
df.to_csv('output.csv')

---

In [144]:
df[df['id'].isin([52, 56, 63, 84, 118, 122, 169, 171, 174])]

,id,barcode,name,cleaned,num_words
116,118,7891463000866,ABOCATE (ANGIOCATH) INTRAV16,ABOCATE INTRAV16,4
120,122,7891338700105,ACTISORB PLUS 10.5X10.5CM MAP105,ACTISORB PLUS 10.5X10.5CM MAP105,4
167,169,7898225526923,ADA TINA INPELE CAPELLI FORTI 60 CPS 30G,ADA TINA INPELE CAPELLI FORTI 60 CPS,7
62,63,7891962004433,"AMANTEIGADO BANANA BAUDUCCO 11,5G","AMANTEIGADO BANANA BAUDUCCO 11,5G",4
51,52,7897371801274,CHUP NUK R 729532,CHUP NUK R 729532,4
55,56,7897371800505,CHUP NUK R729244,CHUP NUK R729244,3
83,84,7898312327488,ESC PROART BAMBU REF 73252,ESC PROART BAMBU 73252,5
169,171,7891040089956,"NEXCARE ESPARADRAPO IMPERMEAVEL 1,2CMX3M","NEXCARE ESPARADRAPO IMPERMEAVEL 1,2CMX3M",4
172,174,7897262419960,PORTA RETRATOS 10CMX15CM,PORTA RETRATOS 10CMX15CM,3


In [71]:
# dict_ = {
#     'SAB': 'SABONETE',
#     'DES': 'DESODORANTE',
#     'ESC': 'ESCOVA',
#     'LOC': 'LOCAO',
#     'TRAD': 'TRADICIONAL',
#     'CURAT': 'CURATIVO',
# }